# Bioinformatics Algorithms, Week 3

This week, we are switching gears from looking for the *DnaA* boxes in bacterial genomes, to looking for **regulatory motifs** responsible for maintaining circadian rhythms. Regulatory motifs are short nucleotide sequences, typically upstream of genes, which are bound by transcription factors that may activate or repress transcription. 

(A promoter a kind of a regulatory motif in DNA; activators and repressors are proteins. A *DnaA* box is not a promoter, but it is a regulatory motif found near the promoter. DNA polymerase is not an activator). 

### Identifying the evening element
In [Harmer et al. (2000)](https://doi.org/10.1126/science.290.5499.2110), Steve Kay's team used a DNA microarray to identify genes expressed at different times of the day (transcriptomes). They then extracted the upstream regions (L = 1000 bps) of 500 genes that exhibited circadian behavior, and looked for Frequent Words. They found the sequence AAAATATCT 46 times. The expected number of occurrences of a 9-mer in 500 random DNA strings, each of length 1000 bps is 1.892, since:

- The probability of randomly generating any 9-mer is $0.25^9$.
- In a 1000 bp sequence, the 9-mer can occupy $992$ (i.e. $1000-9+1$) positions.
- There are $500$ such sequences.
- Therefore, $0.25^9 \times 992 \times 500 = 1.89208984375$ is the expected number.

So it turns out that the sequence AAAATATCT was **about 24 times more frequent** than expected. Harmer et al. (2000) named this promoter sequence the **evening element** and verified that it does, indeed, govern circadian behavior. The idea is that a single activator, or a set of activators that recognize the same promoter AAAATATCT, will activate the transcription of all 500 genes associated with circadian behavior at the same time in a plant, so that all necessary proteins are available for the plant to express circadian behavior.  

Perhaps because maintaining a correct circadian rhythm is a matter of life or death for plants, the evening element is remarkably well-conserved in plants. However, regulatory motifs in other genes and organisms may not be so conserved. Fruit fly **immunity genes** have very similar, but not identical, 12-mer promoters, where a transcription factor called **NF-kB** binds and activates transcription. 

```
1   T C G G G G g T T T t t
2   c C G G t G A c T T a C
3   a C G G G G A T T T t C
4   T t G G G G A c T T t t
5   a a G G G G A c T T C C
6   T t G G G G A c T T C C
7   T C G G G G A T T c a t
8   T C G G G G A T T c C t
9   T a G G G G A a c T a C
10  T C G G G t A T a a C C
```

The uppercase letters indicate the most common nucleotide in each column.

When we have a case like this, where there are numerous (>2) differences between strings and our *k*-mers get quite long, the **Frequent Words** solution that we spent the last two weeks developing will no longer help us because it will be too slow. Moreover, unlike *DnaA* boxes, which occur in clumps near the *ori*, regulatory motifs tend to be scattered throughout the genome, which means we might be dealing with windows that are tens or hundreds of thousands of base pairs wide. We need another approach.

### 1. Motif Enumeration (Implanted Motif Problem)
We can first try a brute-force, **exhaustive search** which considers every possible candidate and checks whether each candidate solves the problem. While such a search is guaranteed to return a correct solution, its time complexity makes it an NP problem, and it is not guaranteed to return an answer in a reasonable amount of time. The function below does not entirely follow the pseudocode given in the course, because I could not make sense of the pseudocode.

**Implanted Motif Problem**: *Find all (k, d)-motifs in a collection of strings.*
- **Input**: A collection of strings *Dna*, and integers *k* and *d*.
- **Output**: All (*k*, *d*)-motifs, which are *k*-mers appearing in every string in *Dna* with at most *d* mismatches.


In [1]:
def hamming_distance(s1, s2):
    diff = [i for i in range(len(s1)) if s1[i] != s2[i]]
    return len(diff)

def neighbors(pattern, d):
    if d == 0:
        return {pattern}
    if len(pattern) == 1:
        return {"A", "C", "G", "T"}
    neighborhood = set()
    suffix_neighbors = neighbors(pattern[1:], d)
    for text in suffix_neighbors:
        if hamming_distance(pattern[1:], text) < d:
            nts = ["A", "C", "G", "T"]
            for nt in nts:
                neighborhood.add(nt+text)
        else:
            neighborhood.add(pattern[0]+text)
    return neighborhood
    
def kmer_in_all_dna(kmer, kmers_dict):
    for array in kmers_dict.values():
        if kmer not in array:
            return False
    return True
    
def motif_enumeration(dna, k, d):
    patterns = set()
    kmers_dict = {}
    all_kmers = set()

    for pattern in dna:
        kmers = {pattern[i:i+k] for i in range(len(pattern)-k+1)}
        kmers_dict[pattern] = set()
        for kmer in kmers:
            kmer_neighbors = neighbors(kmer, d)
            for i in kmer_neighbors:
                kmers_dict[pattern].add(i) # d-neighbors of the k-mer
                all_kmers.add(i)
            kmers_dict[pattern].add(kmer) # the k-mer itself
            all_kmers.add(kmer)
                        
    for kmer in all_kmers:
        if kmer_in_all_dna(kmer, kmers_dict):
            patterns.add(kmer)
    
    return patterns

In [2]:
dna = ["ATTTGGC", "TGCCTTA", "CGGTATC", "GAAAATT"]
k = 3 
d = 1
motif_enumeration(dna, k, d)

{'ATA', 'ATT', 'GTT', 'TTT'}

### 2a. Motif Scoring
The **Implanted Motif** problem has some limitations. If even a single string in *Dna* does not contain the regulatory motif, a (*k*, *d*)-motif does not exist at all. 

A different way to think about the problem might be to score individual instances of the motif depending on how similar they are to an "ideal" motif. But since the ideal motif is unknown, we select a *k*-mer from each string, and score these *k*-mers based on how similar they are to **each other**.

Given a list of *t* strings *Dna*, where each string has a length *n*, and we want to score *k*-mers. We will be dealing with a $t \times k$ matrix *Motifs*.

```
               k

   [[T C G G G G g T T T t t]
    [c C G G t G A c T T a C]
    [a C G G G G A T T T t C]
    [T t G G G G A c T T t t]
t   [a a G G G G A c T T C C]
    [T t G G G G A c T T C C]
    [T C G G G G A T T c a t]
    [T C G G G G A T T c C t]
    [T a G G G G A a c T a C]
    [T C G G G t A T a a C C]]
```

By varying the choice of *k*-mers from each string in *Dna*, we can construct a large number of different *Motif* matrices. Our goal is to obtain the most "conserved" matrix, with the least amount of differences across the *k*-mers in the matrix. To quantify the similarity of the *k*-mers, we can simply count the number of nucleotides at each position in *k* that are not in the plurality. 

In [3]:
def score(motifs):
    k = len(motifs[0])
    
    score_dicts = []
    for i in range(k):
        nt_counts = {"A":0, "T":0, "G":0, "C":0}
        for motif in motifs:
            nt_counts[motif[i]] += 1
        score_dicts.append(nt_counts)

    scores = []
    for d in score_dicts:
        dict_score = 0
        for key, value in d.items():
            if key != max(d, key=d.get):
                dict_score += value
        scores.append(dict_score)
    return sum(scores)

In [4]:
dna = ["TCGGGGGTTTTT",
    "CCGGTGACTTAC",
    "ACGGGGATTTTC",
    "TTGGGGACTTTT",
    "AAGGGGACTTCC",
    "TTGGGGACTTCC",
    "TCGGGGATTCAT",
    "TCGGGGATTCCT",
    "TAGGGGAACTAC",
    "TCGGGTATAACC"]

score(dna)

30

But, currently, we ignore the difference between a column having 6C, 2A, and 2T versus a column having 6C and 4T. Both end up with the score of 6, yet the latter is actually more conserved than the former! This is important because many regulatory motifs have a few "strongly conserved" positions and many "weakly conserved" positions that allow one of several nucleotides with no clear preference for one over the other. 

To account for the degree to which having the "right" nucleotide matters, we turn to **entropy**. It is a measure of uncertainty for probability distributions, and is defined as: 

$$ H(p_1, \dots, p_N) = -\Sigma^N_{i=1} (p_i\times\log_2{p_i})$$

In [5]:
import math
import numpy as np

def count(motifs):
    k = len(motifs[0])
    counts = [[0]*k for i in range(4)] # empty matrix to fill
    nt_index_map = {"A": 0, "C": 1, "G": 2, "T": 3}
    for motif in motifs:
        for i in range(k):
            nt = motif[i]
            counts[nt_index_map[nt]][i] += 1
    return counts

def profile(motifs):
    counts = np.array(count(motifs))
    return counts/len(motifs)

def safe_log2(value):
    try:
        return math.log2(value)
    except: 
        return 0 # log(0) is undefined but we need to return 0
        
def entropy(row):
    plogp = [value*safe_log2(value) for value in row]
    ent = -1*sum(plogp)
    return ent

def total_entropy(matrix):
    pf = profile(matrix)
    entropies = [ entropy(row) for row in pf.T ]
    return sum(entropies)

In [6]:
total_entropy(dna)

9.916290005356972

Entropy offers an improved method of scoring motif matrices than just counting the non-plurality nucleotides, and in practice entropy is used more often, but for simplicity's sake, we'll use the **score** function instead.

We can also obtain a **count** matrix, and if we divide all elements in **count** by *t*, the number of strings in *Dna*, we obtain the **profile** matrix.

In [7]:
import numpy as np

def count(motifs):
    k = len(motifs[0])
    counts = [[0]*k for i in range(4)] # empty matrix to fill
    nt_index_map = {"A": 0, "C": 1, "G": 2, "T": 3}
    for motif in motifs:
        for i in range(k):
            nt = motif[i]
            counts[nt_index_map[nt]][i] += 1
    return counts

def profile(motifs):
    counts = np.array(count(motifs))
    return counts/len(motifs)

In [8]:
profile(dna)

array([[0.2, 0.2, 0. , 0. , 0. , 0. , 0.9, 0.1, 0.1, 0.1, 0.3, 0. ],
       [0.1, 0.6, 0. , 0. , 0. , 0. , 0. , 0.4, 0.1, 0.2, 0.4, 0.6],
       [0. , 0. , 1. , 1. , 0.9, 0.9, 0.1, 0. , 0. , 0. , 0. , 0. ],
       [0.7, 0.2, 0. , 0. , 0.1, 0.1, 0. , 0.5, 0.8, 0.7, 0.3, 0.4]])

Finally, using the **profile** matrix, we can find out what the **consensus string** is. 

In [9]:
def consensus(motifs):
    pf = profile(motifs)
    index_nt_map = {0: "A", 1: "C", 2: "G", 3: "T"}
    max_values = np.argmax(pf, axis=0)
    
    return "".join([index_nt_map[i] for i in max_values])

In [10]:
consensus(dna)

'TCGGGGATTTCC'

Super cool! The consensus string is not actually found in any of the 10 strings in *Dna*, but it still counts. 

### 2b. The Motif Finding Problem

Now that we've learned how to evaluate a collection of *k*-mers, we can formulate the **Motif Finding** problem.

**Motif Finding Problem**: *Given a collection of strings, find a set of k-mers, one from each string, that minimizes the score of the resulting motif.*
- **Input**: A collection of strings *Dna* and an integer *k*.
- **Output**: A collection **Motifs** of *k*-mers, one from each string in *Dna*, minimizing *Score*(*Motifs*) among all possible choices of *k*-mers.

So, this is an optimization problem. A brute force algorithm would be way too slow (O(n<sup>k</sup>)), so we have to figure out a faster algorithm. Given the same matrix as before:

```
               k

   [[T C G G G G g T T T t t]
    [c C G G t G A c T T a C]
    [a C G G G G A T T T t C]
    [T t G G G G A c T T t t]
t   [a a G G G G A c T T C C]
    [T t G G G G A c T T C C]
    [T C G G G G A T T c a t]
    [T C G G G G A T T c C t]
    [T a G G G G A a c T a C]
    [T C G G G t A T a a C C]]
```

We can actually calculate score by tallying up the column-consensus nucleotide per row (in this matrix, it would be the capitalized letters). It also happens that the score per row is just the Hamming distance between the row and the consensus string for this matrix, and so the overall *Score*(*Motifs*) equals the sum of Hamming distances.

<img src="http://bioinformaticsalgorithms.com/images/Motifs/motifs_score_consensus.png" width=700px />

Alright. The instructors define a **function *d*(*Pattern*, *Motifs*) as the sum of Hamming distances between *Pattern* and *Motif*** in *Dna*. And because *Score*(*Motifs*) = *d*(*Pattern*, *Motifs*), rather than looking for a set *Motifs* that minimizes *Score*, we can look for a string *Pattern* that minimizes *d* given all possible sets *Motifs* in *Dna*.

**Equivalent Motif Finding Problem**: *Given a collection of strings, find a collection of k-mers (one from each string) that minimizes the distance between all possible patterns and all possible collections of k-mers.*
- **Input**: A collection of strings *Dna* and an integer *k*.
- **Output**: A *k*-mer *Pattern* and a collection of *k*-mers, one from each string in *Dna*, minimizing *d*(*Pattern*, *Motifs*) among all possible choices of *Pattern* and *Motifs*.

### 3. The Median String Problem: A reformulation of the Motif Finding Problem

Well, before, we had to look for a set *Motifs*, but now, we have to look for a string *Pattern* in addition to a set *Motifs*. Haven't we made our job more difficult? The insight that makes the **Equivalent Motif Finding** approach better is that, given *Pattern*, we don't have to explore all possible sets *Motifs*.

We define a new set *Motifs*(*Pattern*, *Dna*) as one set among all possible sets of *Motifs* in *Dna* that minimizes *d*(*Pattern*, *Motifs*) for a given *Pattern*. For example, given *Pattern*=AAA, and a collection of strings *Dna* below, the set *Motifs*(*Pattern*, *Dna*) with the lowest sum of Hamming distances from AAA is {AAC, ATA, ACG, AAA, AGA}.

```
       [[ttaccttAAC],
        [gATAtctgtc],
Dna =   [ACGgcgttcg],
        [ccctAAAgag],
        [cgtcAGAggt]]
```

And we don't have to consider every possible set of *Motifs* in *Dna* because we can simply consider each string in *Dna* and find the *k*-mer that returns the minimum distance in *Hamming*(*Pattern*, *k*-mer). So rather than generating all possible sets *Motifs* given *Dna*, which would be monstrously many given anything but a toy problem, we just consider each string in *Dna* and get the distance-minimizing *k*-mer. For example, we'd run: *Motif*(GATTCTCA, GCAAAGACGCTGACCAA) = GACGCTGA.

To be clear, the advantage of the **Median String** formulation over **Motif Finding** is that, rather than iterating through every possible set of *Motifs*, we just iterate through every possible *k*-mer *Pattern'*. The **Motif Finding** solution ends up with a runtime of O(n<sup>t</sup>) where t = number of strings in *Dna*, and **Median String** ends up with a runtime of O(4<sup>k</sup>) where k = the length of the *k*-mer sought. Both are polynomial time, but the number of strings in *Dna* number in the thousands in practice while *k* does not typically exceed 20 bps. So solving the **Median String** problem improves runtime significantly!

If we define the minimum possible distance between *Pattern* and a string in *Dna* as:

$$d(Pattern, Text) = \text{min}_\text{all kmers Pattern' in Text} HammingDistance(Pattern, Pattern') $$

Then we can define the minimum distance between *Pattern* and a set of strings *Dna* as:

$$d(Pattern, Dna) = \Sigma_{i=1}^{t} d(Pattern, Dna_i) $$

In [11]:
def hamming_distance(s1, s2):
    diff = [i for i in range(len(s1)) if s1[i] != s2[i]]
    return len(diff)


def distance_between_pattern_and_strings(pattern, dna):
    k = len(pattern)
    distance = 0
    for text in dna:
        hamming_dist = float('inf')
        
        # This block finds the minimum d(pattern, text)
        for i in range(len(text)-k+1):
            pattern_prime = text[i:i+k]
            if hamming_dist > hamming_distance(pattern, pattern_prime):
                hamming_dist = hamming_distance(pattern, pattern_prime)
        distance += hamming_dist
    return distance

In [12]:
distance_between_pattern_and_strings("GA", ["GATT", "CGAT", "TTGA"])

0

And the string that minimizes *d*(*Pattern*, *Dna*) is the **median string**.

**Median String Problem**: *Find a median string.*
- **Input**: A collection of strings *Dna* and an integer *k*.
- **Output**: A *k*-mer *Pattern* that minimizes *d*(*Pattern*, *Dna*) among all possible choices of *k*-mers.

In [13]:
def neighbors(pattern, d):
    if d == 0:
        return {pattern}
    if len(pattern) == 1:
        return {"A", "C", "G", "T"}
    neighborhood = set()
    suffix_neighbors = neighbors(pattern[1:], d)
    for text in suffix_neighbors:
        if hamming_distance(pattern[1:], text) < d:
            nts = ["A", "C", "G", "T"]
            for nt in nts:
                neighborhood.add(nt+text)
        else:
            neighborhood.add(pattern[0]+text)
    return neighborhood

def median_string(dna, k):
    distance = float('inf')
    all_kmers = neighbors("".join(["A" for i in range(k)]), k)
    for pattern in all_kmers:
        if distance > distance_between_pattern_and_strings(pattern, dna):
            distance = distance_between_pattern_and_strings(pattern, dna)
            median = pattern
    return median

In [14]:
dna = ["AAATTGACGCAT", "GACGACCACGTT", "CGTCAGCGCCTG", "GCTGAGCACCGG", "AGTTCGGGACAG"]
median_string(dna, 3)

'GAC'

The **Median String** problem is actually not exactly equivalent to the **Motif Finding** problem, because the former returns a single median string while the latter returns a set of motifs. I don't know why the authors don't address this point. 

In any case, even though solving the **Median String** is much faster than solving the **Motif Finding** problem, the former is still not practical in real-world applications because as *k* increases, it becomes much too slow. The authors say that solving for a *k*=13 median string took half a day. We will be searching for a 20 bp regulatory motif, and using our **Median String** will not be feasible. So, we'll have to modify our approach once again.

### 4. Greedy Motif Search

**Greedy algorithms** are a class of algorithms that choose the most attractive (i.e. probabilistically most likely) options at each iteration. While fast, greedy algorithms tend to return only approximate solutions. In essence, we trade accuracy for speed.

Earlier, we learned how to create a matrix of probabilities or a **profile** of a set *Motifs*. Using the profile, we can find the most likely *k*-mer in any given string of nucleotides. This will be handy in implementing a greedy solution to the **Motif Finding** problem.

**Profile-most Probable *k*-mer Problem**: Find a *Profile*-most probable *k*-mer in a string.

- **Input**: A string *Text*, an integer *k*, and a 4 × *k* matrix *Profile*.
- **Output**: A *Profile*-most probable *k*-mer in *Text*.

In [15]:
def profile_most_probable_kmer(text, k, profile):
    char_index_dict = {"A": 0, "C": 1, "G": 2, "T": 3}
    
    most_probable_kmer = ""
    max_probability = float('-inf')
    for i in range(len(text)-k+1):
        kmer = text[i:i+k]
        current_prob = 1
        for j in range(k):
            nt = char_index_dict[kmer[j]]
            current_prob *= profile[nt][j]
        if current_prob > max_probability:
            max_probability = current_prob
            most_probable_kmer = kmer
    return most_probable_kmer

In [16]:
t = "ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT"
k = 5
p = [[0.2, 0.2, 0.3, 0.2, 0.3],
    [0.4, 0.3, 0.1, 0.5, 0.1],
    [0.3, 0.3, 0.5, 0.2, 0.4],
    [0.1, 0.2, 0.1, 0.1, 0.2]]
profile_most_probable_kmer(t, k, p)

'CCGAG'

The **Greedy Motif Search** pseudocode given in the course is barely comprehensible. I explain it as follows:

1. We create an initial *BestMotifs* matrix by arbitrarily choosing the first *k*-mer in each string in *Dna*.
2. We compare len(dna)-k+1 sets of *Motifs* as follows, iterating with counter *i*:

    2.1. Initialize the set *Motifs* with just a motif at position *i* from the first string in *Dna*.
    
    2.2. Calculate the *Profile*.
    
    2.3. Based on the *Profile*, find the best motif in the next string in *Dna*. Add the motif to *Motifs*.
    
    2.4. Repeat until a motif has been found in all strings in *Dna*. 
    
    2.5. Calculate the score of the resulting *Motifs*. If the score of *Motifs* is better than the arbitrarily created *BestMotifs*, then make the new set *Motifs* into *BestMotifs*.
    
    2.6. Repeat until all len(dna)-k+1 sets of *Motifs* have been compared.
    
3. Return *BestMotifs*.
    

**GreedyMotifSearch**.
- **Input**: Integers *k* and *t*, followed by a space-separated collection of strings *Dna*.
- **Output**: A collection of strings *BestMotifs* resulting from applying *GreedyMotifSearch*(*Dna*, *k*, *t*). If at any step you find more than one *Profile*-most probable *k*-mer in a given string, use the one occurring first.


In [17]:
def greedy_motif_search(k, t, dna):
    
    # artitrarily choose first k chars from each dna string to create an initial Motifs matrix.
    best_motifs = [text[:k] for text in dna] 
    
    for i in range(len(dna[0])-k+1):
        
        motif_1 = dna[0][i:i+k]
        
        motifs = [motif_1] # initialize a Motifs list for each motif.
        
        for j in range(1, t): # for all strings in Dna excluding the first,
            
            # get the probability profile given Motifs
            # profile is updated over for loop iterations
            # based on dna[1] to dna[t]
            prof = profile(motifs) 
            
            # and find the most probable k-mer in the dna string given profile
            motif_i = profile_most_probable_kmer(dna[j], k, prof) 
            
            # and update Motifs
            # in the i+1 loop, information from motif_i will be used
            # to generate a better profile
            motifs.append(motif_i) 
            
        # if Motifs has better score
        if score(motifs) < score(best_motifs): 
            best_motifs = motifs
            
    return best_motifs

In [18]:
k = 3
t = 5
dnas = ["GGCGTTCAGGCA", "AAGAATCAGTCA", "CAAGGAGTTCGC", "CACGTCAATCAC", "CAATAATATTCG"]

print(" ".join(greedy_motif_search(k, t, dnas)))

CAG CAG CAA CAA CAA


### Greedy Motif Search with Pseudocounts

Compared to **Median String**, **Greedy Motif Search** is quite fast, even at k = 15, and returns an answer. However, it trades speed for accuracy and performs, actually, quite poorly. This is because the initial matrix *Profile* contains many zeros, and because multiplying anything with zero yields a zero, this represents a serious problem. For instance, even though the string TCG**T**GATTTCC is only one nucleotide off from the consensus string TCG**G**GATTTCC at the fourth position, the probability of TCG**T**GATTTCC given the *Profile* constructed using our usual method equals 0. 

<img src='http://bioinformaticsalgorithms.com/images/Motifs/greedy_profile_1.png' width=700px />



Oliver Cromwell, in attempting to persuade the Church of Scotland to reconsider proclaiming Charles II as king, wrote:

> I beseech you, in the bowels of Christ, think it possible that you may be mistaken.

**Cromwell’s rule** states that we should not use probabilities of 0 or 1 unless we are talking about logical statements that can only be true or false. 

Following this dictum, bioinformaticians replace zeroes with **pseudocounts** to avoid unfair scores. The simplest is **Laplace's Rule of Succession**, which, in our application, adds one to each element, and to calculate *Profile*, divides each element by *t*+4 rather than simply *t* (+4 comes from there being four kinds of nucleotides). 

In [23]:
def laplace_profile(motifs):
    counts = np.array(count(motifs))
    return (counts+1)/(len(motifs)+4)


def greedy_motif_search_with_pseudocounts(k, t, dna):
    
    best_motifs = [text[:k] for text in dna] 
    
    for i in range(len(dna[0])-k+1):
        motif_1 = dna[0][i:i+k]
        motifs = [motif_1] 
        
        for j in range(1, t): 
            prof = laplace_profile(motifs) 
            motif_i = profile_most_probable_kmer(dna[j], k, prof) 
            motifs.append(motif_i) 
    
        if score(motifs) < score(best_motifs): 
            best_motifs = motifs
            
    return best_motifs

In [25]:
k = 3 
t = 5
dna = ["GGCGTTCAGGCA", "AAGAATCAGTCA", "CAAGGAGTTCGC", "CACGTCAATCAC", "CAATAATATTCG"]
greedy_motif_search_with_pseudocounts(k, t, dna)

['TTC', 'ATC', 'TTC', 'ATC', 'TTC']

Quite different results given the same input. Using pseudocounts gives us a much better result, but it is possible to design an even better algorithm. We'll tackle that in Week 4. 